In [1]:
def nmr_file_parser(peak_file_dir):
    file = open(peak_file_dir, 'r')
    c1,c2=[],[]
    h1,h2=[],[]
    cross,reflected=[],[]
    d1,d2=[],[]
    lab=[]
    for line in file:
        data = line.split()
        res1=residue_parser(data[0])
        res2=residue_parser(data[1])
        lab.append(res1+','+res2)
        c1.append(data[2])
        c2.append(data[4])
        h1.append(data[3])
        h2.append(data[5])
        cross.append(data[6])
        reflected.append(data[7])
        d1.append(data[14])
        d2.append(data[15])
    
    return lab,c1,h2,c2,h2,cross,reflected,d1,d2
    
        
def residue_parser(residue_str):
    res_letter = residue_str[0]
    integers = [int(s) for s in re.findall(r'-?\d+\.?\d*', residue_str)]
    out_str = ''
    if(res_letter=='I'):
        out_str='ILE'+' '+str(integers[0])+' A'
    elif(res_letter=='A'):
        out_str='ALA'+' '+str(integers[0])+' A'
    elif(res_letter=='L'):
        if('D1' in residue_str):
            out_str='LEU'+' '+str(integers[0])+' A'
        elif('D2' in residue_str):
            out_str='LEU'+' '+str(integers[0])+' B'
    elif(res_letter=='V'):
        if('G1' in residue_str):
            out_str='VAL'+' '+str(integers[0])+' A'
        elif('G2' in residue_str):
            out_str='VAL'+' '+str(integers[0])+' B'
    if(out_str==''):
        print 'BLANK AMINO ACID CODE. AMINO ACID STRING TO BE PARSED = ',residue_str
        sys.exit(100)
    return out_str


peak_file_dir='input/correlate.3'
lab,c1,h2,c2,h2,cross,reflected,d1,d2=nmr_file_parser(peak_file_dir)



IOError: [Errno 2] No such file or directory: 'input/correlate.3'

In [92]:
def nmr_masker(h1,h2,c1,c2,threshold=0.01):
    #Takes in 4 2D square matrices. Will then calculate the difference between peaks, in an expanded 4D matrix. 
    #If two peaks are too close, IE closer than the threshold level, we eliminate them from the data set
    #IE we mask them from the 2D square matrix, setting the correspond indices to True in the mask
    
    #SETUP MATRICES TO STORE DATA:
    num=h1.shape[0]
    mask=numpy.zeros((num,num),dtype=bool)
    diff_matrix=numpy.zeros((num,num,num,num))
    
    #SETUP INDEXING MATRICES:
    i,j,k,l=numpy.arange(num),numpy.arange(num),numpy.arange(num),numpy.arange(num)
    ii,jj,kk,ll=numpy.meshgrid(i,j,k,l)
    
    #EXPAND TO A 4D DIFFERENCE MATRIX - TAKES DIFFERENCE BETWEEN PEAK (ii,jj) and peak (kk,ll):
    diff_matrix[ii,jj,kk,ll]=2.*numpy.sqrt(((h1[ii,jj]-h1[kk,ll])/(h1[ii,jj]+h1[kk,ll]))**2 + ((h2[ii,jj]-h2[kk,ll])/(h2[ii,jj]+h2[kk,ll]))**2 + ((c1[ii,jj]-c1[kk,ll])/(c1[ii,jj]+c1[kk,ll]))**2 + ((c2[ii,jj]-c2[kk,ll])/(c2[ii,jj]+c2[kk,ll]))**2)
    
    #CUT-OFF AT WHERE THIS DIFFERENCE IS SMALLER THAN THRESHOLD:
    sum_matrix=diff_matrix<threshold
    
    #FIND "COLUMNS" ALONG WHICH THIS SUM IS GREATER THAN EXPECTED 1
    sum_matrix=numpy.sum(sum_matrix,axis=(2,3))
    mask=sum_matrix<2
    return mask

h1,h2=numpy.array([[1.05,1.23],[1.07,1.59]]),numpy.array([[1.05,1.23],[1.05,1.59]])
c1,c2=numpy.array([[10.59,8.39],[10.59,12.31]]),numpy.array([[9.37,8.39],[9.37,12.31]])
mask=nmr_masker(h1,h2,c1,c2)
print 'RAW DATA = \n',h1
print 'MASKED DATA = \n',h1[mask]


RAW DATA = 
[[1.05 1.23]
 [1.07 1.59]]
MASKED DATA = 
[1.05 1.23 1.07 1.59]


In [54]:
def d1_masker(data,threshold=0.01):
    num=data.shape[0]
    i,j=numpy.arange(num),numpy.arange(num)
    ii,jj=numpy.meshgrid(i,j,indexing='ij')
    diff_matrix=numpy.zeros((num,num))
    diff_matrix[ii,jj]=numpy.sqrt(((data[ii]-data[jj])/(data[ii]+data[jj]))**2)
    print 'diff_matrix =\n',diff_matrix
    bool_matrix=diff_matrix<threshold
    print 'bool_matrix = \n',bool_matrix
    sum_arr=numpy.sum(bool_matrix,axis=0)-1
    print 'sum_arr = \n',sum_arr
    bool_arr=sum_arr<1
    print 'bool_arr = \n',bool_arr
    return bool_arr
    
data=numpy.array([1.,1.05,1.03,1.04,1.25,1.59,1.83,2.01])
mask=d1_masker(data)
print 'RAW DATA = \n',data
print 'MASKED DATA = \n',data[mask]

diff_matrix =
[[0.         0.02439024 0.01477833 0.01960784 0.11111111 0.22779923
  0.29328622 0.33554817]
 [0.02439024 0.         0.00961538 0.00478469 0.08695652 0.20454545
  0.27083333 0.31372549]
 [0.01477833 0.00961538 0.         0.00483092 0.09649123 0.21374046
  0.27972028 0.32236842]
 [0.01960784 0.00478469 0.00483092 0.         0.09170306 0.20912548
  0.27526132 0.31803279]
 [0.11111111 0.08695652 0.09649123 0.09170306 0.         0.11971831
  0.18831169 0.23312883]
 [0.22779923 0.20454545 0.21374046 0.20912548 0.11971831 0.
  0.07017544 0.11666667]
 [0.29328622 0.27083333 0.27972028 0.27526132 0.18831169 0.07017544
  0.         0.046875  ]
 [0.33554817 0.31372549 0.32236842 0.31803279 0.23312883 0.11666667
  0.046875   0.        ]]
bool_matrix = 
[[ True False False False False False False False]
 [False  True  True  True False False False False]
 [False  True  True  True False False False False]
 [False  True  True  True False False False False]
 [False False False False  Tru

In [93]:
def nmr_masker(h1,h2,c1,c2,threshold=0.01):
    #Takes in 4 2D square matrices. Will then calculate the difference between peaks, in an expanded 4D matrix. 
    #If two peaks are too close, IE closer than the threshold level, we eliminate them from the data set
    #IE we mask them from the 2D square matrix, setting the correspond indices to True in the mask
    
    #SETUP MATRICES TO STORE DATA:
    num=h1.shape[0]
    mask=numpy.zeros((num,num),dtype=bool)
    diff_matrix=numpy.zeros((num,num,num,num))
    
    #SETUP INDEXING MATRICES:
    i,j,k,l=numpy.arange(num),numpy.arange(num),numpy.arange(num),numpy.arange(num)
    ii,jj,kk,ll=numpy.meshgrid(i,j,k,l)
    
    #EXPAND TO A 4D DIFFERENCE MATRIX - TAKES DIFFERENCE BETWEEN PEAK (ii,jj) and peak (kk,ll):
    diff_matrix[ii,jj,kk,ll]=2.*numpy.sqrt(((h1[ii,jj]-h1[kk,ll])/(h1[ii,jj]+h1[kk,ll]))**2 + ((h2[ii,jj]-h2[kk,ll])/(h2[ii,jj]+h2[kk,ll]))**2 + ((c1[ii,jj]-c1[kk,ll])/(c1[ii,jj]+c1[kk,ll]))**2 + ((c2[ii,jj]-c2[kk,ll])/(c2[ii,jj]+c2[kk,ll]))**2)
    
    #CUT-OFF AT WHERE THIS DIFFERENCE IS SMALLER THAN THRESHOLD:
    sum_matrix=diff_matrix<threshold
    
    #FIND "COLUMNS" ALONG WHICH THIS SUM IS GREATER THAN EXPECTED 1
    sum_matrix=numpy.sum(sum_matrix,axis=(2,3))
    mask=sum_matrix<2
    return mask

def peak_file_parser(peak_file_dir):
    file = open(peak_file_dir, 'r')
    cross,reflected={},{}
    diag_1,diag_2={},{}
    for line in file:
        data = line.split()
        res1 = residue_parser(data[0])
        res2 = residue_parser(data[1])
        v1,v2=data[6],data[7]
        a1,a2=data[14],data[15]
        if not(res1 in cross):
            cross[res1]={}
            reflected[res1]={}
            diag_2[res1]={}
            diag_1[res1]={}
        cross[res1][res2]=v1
        reflected[res1][res2]=v2
        diag_1[res1][res2]=a1
        diag_2[res1][res2]=a2
    map_key,map_index={},{}
    i=0
    for res1 in cross:
        map_key[res1]=i
        map_index[i]=res1
        i+=1
    
    num = len(cross)
    cross_matrix, reflected_matrix, d1_matrix, d2_matrix = numpy.zeros((num,num)),numpy.zeros((num,num)),numpy.zeros((num,num)),numpy.zeros((num,num))
    mask_matrix=numpy.ones((num,num),dtype=bool)
    for res1 in cross:
        for res2 in cross[res1]:
            i1,i2=map_key[res1],map_key[res2]
            cross_matrix[i1,i2]=cross[res1][res2]
            reflected_matrix[i1,i2]=reflected[res1][res2]
            d1_matrix[i1,i2]=diag_1[res1][res2]
            d2_matrix[i1,i2]=diag_2[res1][res2]
            mask_matrix[i1,i2]=False
    
    return cross_matrix,reflected_matrix,d1_matrix,d2_matrix,map_key,map_index,mask_matrix
    
        
def residue_parser(residue_str):
    res_letter = residue_str[0]
    integers = [int(s) for s in re.findall(r'-?\d+\.?\d*', residue_str)]
    out_str = ''
    if(res_letter=='I'):
        out_str='ILE'+' '+str(integers[0])+' A'
    elif(res_letter=='A'):
        out_str='ALA'+' '+str(integers[0])+' A'
    elif(res_letter=='L'):
        if('D1' in residue_str):
            out_str='LEU'+' '+str(integers[0])+' A'
        elif('D2' in residue_str):
            out_str='LEU'+' '+str(integers[0])+' B'
    elif(res_letter=='V'):
        if('G1' in residue_str):
            out_str='VAL'+' '+str(integers[0])+' A'
        elif('G2' in residue_str):
            out_str='VAL'+' '+str(integers[0])+' B'
    if(out_str==''):
        print 'BLANK AMINO ACID CODE. AMINO ACID STRING TO BE PARSED = ',residue_str
        sys.exit(100)
    return out_str


cross,reflected,d1,d2,map_key,map_index,mask_matrix=peak_file_parser('input/correlate.3')



IOError: [Errno 2] No such file or directory: 'input/correlate.3'

In [ ]:
def get_pdb_coord_methyl(pdb_file):
    #Initialize stuff - Set which file to open; initilaize lists for all atoms in file, and then result list - methyl_list
    print pdb_file
    file = open(pdb_file, 'r')
    atom_list = []
    methyl_list = {}
    
    #Iterate over lines in file, and fill atom_list with position, type & residue (type & number) for each atom:
    for line in file:
        if(line[0:4]=='ATOM'):
#             print line
            atom_number = line[6:12]
            atom_name = line[13:16]
            residue_name = line[17:20]
            residue_number = int(line[22:26].strip())
            x_coord = float(line[30:38])
            y_coord = float(line[38:46])
            z_coord = float(line[46:54])
            
            atom = {}
            atom['num']  = atom_number
            atom['ele'] = atom_name
            atom['residue'] = residue_name + ' ' + str(residue_number)
            atom['x'], atom['y'], atom['z'] = x_coord, y_coord, z_coord
            atom_list.append(atom)

#                 print 'ATOM = ', atom_name
#                 print 'RESIDUE = ', residue_name
#                 print 'X = ', x_coord
#                 print 'Y = ', y_coord
#                 print 'Z = ', z_coord
#                 print line
    
    #Now iterate over all atoms in atom_list and group in residue (type & number):
    residue_list = {}
    for atom in atom_list:
        if(atom['residue'] in residue_list):
            residue_list[atom['residue']][atom['ele']]=(atom['x'],atom['y'],atom['z'])
        else:
            residue_list[atom['residue']]={}
            residue_list[atom['residue']][atom['ele']]=(atom['x'],atom['y'],atom['z'])

    #Now iterate over residues and generate methyls from ILE, LEU, VAL:
    #General strategy - Each methyl consits of 3 vectors - c_m, c_a, c_b. These totally specify a methyl groups position & orientation, *including* psi angle
    #REF:
#     def CalcMetGeom(c_m, c_a, c_b, r_m=1.09, theta_m=numpy.arccos(1./3.)):
    residues = []
    count_ile, count_leu, count_val,count_ala = 0,0,0,0
    for key in residue_list:
        residue = residue_list[key]
        if(key[0:3]=='ILE'):
            residues.append(key)
            methyl = CalcMetGeom(residue['CD1'], residue['CG1'], residue['CB '])
            methyl_list[key+' A'] = methyl
            count_ile+=1
        elif(key[0:3]=='LEU'):
            residues.append(key)
            methyl = CalcMetGeom(residue['CD1'], residue['CG '], residue['CB '])
            methyl_list[key + ' A'] = methyl
            methyl = CalcMetGeom(residue['CD2'], residue['CG '], residue['CB '])
            methyl_list[key + ' B'] = methyl
            count_leu+=1
        elif(key[0:3]=='VAL'):
            residues.append(key)
            methyl = CalcMetGeom(residue['CG1'], residue['CB '], residue['CA '])
            methyl_list[key + ' A'] = methyl
            methyl = CalcMetGeom(residue['CG2'], residue['CB '], residue['CA '])
            methyl_list[key + ' B'] = methyl
            count_val+=1
        elif(key[0:3]=='ALA'):
            residues.append(key)
            methyl=CalcMetGeom(residue['CB '],residue['CA '],residue['C  '])
            methyl_list[key+' A']=methyl
            count_ala+=1

    #Want to return: mapping dictionaries from methyl residue name & number (EG. LEU 113 A) to numpy index, and index to name.
    #3 numpy arrays - N*3 array containing coordinates of methyl C (c_m), N*3 for orientation (v_m), and N*3*3 positions of methyl H (sites)
#     methyl = {'loc':c_m, 'orientation':v_m, 'sites':numpy.array(sites)}
    num = len(methyl_list)
    location = numpy.zeros((num, 3))
    orientation = numpy.zeros((num, 3))
    sites = numpy.zeros((num, 3, 3))
    map_key = {}
    map_index = {}
    i = 0 #Index which incremented to slot methyls into numpy arrays
    print 'count_leu = ', count_leu
    print 'count_val = ', count_val
    print 'count_ile = ', count_ile
    print 'count_ala = ', count_ala
    for key in methyl_list:
        location[i, :] = methyl_list[key]['loc']
        orientation[i, :] = methyl_list[key]['orientation']
        sites[i, :, :] = methyl_list[key]['sites']
        map_key[key] = i
        map_index[i] = key
        i+=1
        
    return location, orientation, sites, map_key, map_index, list(set(residues))

def peak_file_parser(peak_file_dir):
    file = open(peak_file_dir, 'r')
    cross,reflected={},{}
    diag_1,diag_2={},{}
    c1_shift,c2_shift={},{}
    h1_shift,h2_shift={},{}
    for line in file:
        data = line.split()
        res1 = residue_parser(data[0])
        res2 = residue_parser(data[1])
        c1,h1=data[2],data[3]
        c2,h2=data[4],data[5]
        v1,v2=data[6],data[7]
        a1,a2=data[14],data[15]
        if not(res1 in cross):
            cross[res1]={}
            reflected[res1]={}
            diag_1[res1]={}
            diag_2[res1]={}
            c1_shift[res1],c2_shift[res1]={},{}
            h1_shift[res1],h2_shift[res1]={},{}
        cross[res1][res2]=v1
        reflected[res1][res2]=v2
        diag_1[res1][res2]=a1
        diag_2[res1][res2]=a2
        c1_shift[res1][res2],c2_shift[res1][res2]=c1,c2
        h1_shift[res1][res2],h2_shift[res1][res2]=h1,h2
    map_key,map_index={},{}
    i=0
    for res1 in cross:
        map_key[res1]=i
        map_index[i]=res1
        i+=1
    
    num = len(cross)
    cross_matrix,reflected_matrix,d1_matrix,d2_matrix=numpy.zeros((num,num)),numpy.zeros((num,num)),numpy.zeros((num,num)),numpy.zeros((num,num))
    c1_matrix,h1_matrix,c2_matrix,h2_matrix=numpy.zeros((num,num)),numpy.zeros((num,num)),numpy.zeros((num,num)),numpy.zeros((num,num))
    mask_matrix=numpy.ones((num,num),dtype=bool)
    pairs=[]
    for res1 in cross:
        for res2 in cross[res1]:
            pairs.append((res1,res2))
            i1,i2=map_key[res1],map_key[res2]
            cross_matrix[i1,i2]=cross[res1][res2]
            reflected_matrix[i1,i2]=reflected[res1][res2]
            d1_matrix[i1,i2]=diag_1[res1][res2]
            d2_matrix[i1,i2]=diag_2[res1][res2]
            c1_matrix[i1,i2],c2_matrix[i1,i2]=c1_shift[res1][res2],c2_shift[res1][res2]
            h1_matrix[i1,i2],h2_matrix[i1,i2]=h1_shift[res1][res2],h2_shift[res1][res2]
            mask_matrix[i1,i2]=False
    
    return cross_matrix,reflected_matrix,d1_matrix,d2_matrix,c1_matrix,h1_matrix,c2_matrix,h2_matrix,map_key,map_index,pairs,mask_matrix

def nmr_masker(h1,h2,c1,c2,map_key,pairs,threshold=0.01):
    #Takes in 4 2D square matrices. Will then calculate the difference between peaks, in an expanded 4D matrix. 
    #If two peaks are too close, IE closer than the threshold level, we eliminate them from the data set
    #IE we mask them from the 2D square matrix, setting the correspond indices to True in the mask
    
    #SETUP MATRICES TO STORE DATA:
    num=h1.shape[0]
    mask=numpy.zeros((num,num),dtype=bool)
    i_m,j_m=[],[]
    for peak1 in pairs:
        for peak2 in pairs:
            i,j,k,l=map_key[peak1[0]],map_key[peak1[1]],map_key[peak2[0]],map_key[peak2[1]]
            diff=2.*numpy.sqrt(((h1[i,j]-h1[k,l])/(h1[i,j]+h1[k,l]))**2 + ((h2[i,j]-h2[k,l])/(h2[i,j]+h2[k,l]))**2 + ((c1[i,j]-c1[k,l])/(c1[i,j]+c1[k,l]))**2 + ((c2[i,j]-c2[k,l])/(c2[i,j]+c2[k,l]))**2)
            if (diff<threshold) and (peak1[0]!=peak2[0]) and (peak1[1]!=peak2[1]):
                i_m.append(i)
                i_m.append(k)
                j_m.append(j)
                j_m.append(l)
#                 mask[i,j],mask[k,l]=True,True
#                 print 'RESIDUES WHICH ARE MASKED ARE = '
#                 print peak1,': ',h1[i,j],c1[i,j],h2[i,j],c2[i,j],'DIST = ',dist[i,j]
#                 print peak2,': ',h1[k,l],c1[k,l],h2[k,l],c2[k,l],'DIST = ',dist[k,l]
#                 print ' DIFF = ',diff,'\n'
    i_m,j_m=numpy.array(i_m),numpy.array(j_m)
    ii,jj=numpy.meshgrid(i_m,j_m,indexing='ij')
    mask[ii,jj]=True
    return mask
        
def residue_parser(residue_str):
    res_letter = residue_str[0]
    integers = [int(s) for s in re.findall(r'-?\d+\.?\d*', residue_str)]
    out_str = ''
    if(res_letter=='I'):
        out_str='ILE'+' '+str(integers[0])+' A'
    elif(res_letter=='A'):
        out_str='ALA'+' '+str(integers[0])+' A'
    elif(res_letter=='L'):
        if('D1' in residue_str):
            out_str='LEU'+' '+str(integers[0])+' A'
        elif('D2' in residue_str):
            out_str='LEU'+' '+str(integers[0])+' B'
    elif(res_letter=='V'):
        if('G1' in residue_str):
            out_str='VAL'+' '+str(integers[0])+' A'
        elif('G2' in residue_str):
            out_str='VAL'+' '+str(integers[0])+' B'
    if(out_str==''):
        print 'BLANK AMINO ACID CODE. AMINO ACID STRING TO BE PARSED = ',residue_str
        sys.exit(100)
    return out_str

peak_file_dir='input/correlate.3'
cross,reflected,d1,d2,c1,h1,c2,h2,map_key_nmr,map_index_nmr,pairs,mask=peak_file_parser(peak_file_dir)
pdb_file_dir='input/1EZA.pdb'
location,orientation,sites,map_key_xrd,map_index_xrd,res_list=get_pdb_coord_methyl(pdb_file_dir)
i,j=numpy.arange(location.shape[0]),numpy.arange(location.shape[0])
ii,jj=numpy.meshgrid(i,j,indexing='ij')
v_mm = location[jj, :] - location[ii, :]
dist=numpy.sqrt(numpy.sum(v_mm*v_mm,axis=2))
mask1=nmr_masker(h1,h2,c1,c2,map_key_nmr,pairs,threshold=0.10)
mask1=numpy.logical_not(mask1)
mask=numpy.logical_not(mask)
print numpy.sum(mask)
mask=numpy.logical_and(mask,mask1)
print numpy.sum(mask)


#NOW CONVERT INDICES FROM XRD TO NMR:
xrd_indices,nmr_indices=[],[]
for key in map_key_nmr:
    xrd_indices.append(map_key_xrd[key])
    nmr_indices.append(map_key_nmr[key])

xrd_indices,nmr_indices=numpy.array(xrd_indices),numpy.array(nmr_indices)
ii_xrd,jj_xrd=numpy.meshgrid(xrd_indices,xrd_indices,indexing='ij')
ii_nmr,jj_nmr=numpy.meshgrid(nmr_indices,nmr_indices,indexing='ij')

print dist[ii_xrd,jj_xrd][mask[ii_nmr,jj_nmr]]
plt.scatter(h1,h2)
plt.show()
plt.scatter(h1[ii_nmr,jj_nmr][mask[ii_nmr,jj_nmr]],h2[ii_nmr,jj_nmr][mask[ii_nmr,jj_nmr]])
plt.scatter(0,0)
plt.show()
plt.scatter(h1,h2)
plt.scatter(h1[ii_nmr,jj_nmr][mask[ii_nmr,jj_nmr]],h2[ii_nmr,jj_nmr][mask[ii_nmr,jj_nmr]])
plt.show()




